# Mapping vessel ownership
In this notebook, we draw on a very small subset of Global Fishing Watch's fishing effort data and use vessel ownership information from the Western and Central Pacific Fisheries Commission (WCPFC) to demonstrate: 1) the viability of analyzing fishing patterns by corporate structure rather than flag country; 2) that corporations account for a significant share of fishing effort in some locations.

Please run each block of code sequentially by clicking on the "Play" button.

## Import extra libraries of code to help us do our analysis.

In [10]:
library(tidyr)
library(readr)
library(plyr)
library(dplyr)
#library(stringr)
library(rgdal)
library(ggplot2)
library(sf)

ERROR: Error in library(rgdal): there is no package called ‘rgdal’


## Import GFW's fishing effort data 
Described here: https://globalfishingwatch.org/datasets-and-code/fishing-effort/

In [7]:
daily<-read_csv("https://raw.githubusercontent.com/ericnost/gfw/setup/data/2016-12-22.csv")

Parsed with column specification:
cols(
  date = col_date(format = ""),
  lat_bin = col_integer(),
  lon_bin = col_integer(),
  mmsi = col_integer(),
  fishing_hours = col_double(),
  lat = col_double(),
  long = col_double(),
  latlong = col_character()
)


# Load shapefiles

In [8]:
# Countries of the world shapefile from Natural Earth
countries <- read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/countries.geojson")

# WCPFC shapefile from: http://www.fao.org/geonetwork/srv/en/main.home?uuid=cc7dbf20-1b8b-11dd-8bbb-0017f293bd28
wcpfc_map <- read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/wcpfc.geojson") 

# SPC shapefile from: http://www.fao.org/geonetwork/srv/en/main.home?uuid=cc7dbf20-1b8b-11dd-8bbb-0017f293bd28
spc <- read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/spc.geojson")

ERROR: Error in read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/countries.geojson"): could not find function "read_sf"


## Show a basic map of where we'll look at fishing ownership patterns

In [ ]:
map = ggplot() + 
geom_sf(data = wcpfc_map, aes(fill = blue)) +
geom_sf(data = spc, aes(fill = green)) +
geom_sf(data = countries, aes(fill = NAME)) +
theme(legend.position = "none") 
map

In [9]:

# Countries of the world shapefile from Natural Earth
countries <- read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/countries.geojson")#readOGR(dsn="https://raw.githubusercontent.com/ericnost/gfw/setup/data/countries.geojson")
plot(countries, border="wheat3", col="wheat1")

# WCPFC shapefile from: http://www.fao.org/geonetwork/srv/en/main.home?uuid=cc7dbf20-1b8b-11dd-8bbb-0017f293bd28
wcpfc_map <- readOGR(dsn="https://raw.githubusercontent.com/ericnost/gfw/setup/data/wcpfc.geojson") #<- readOGR(dsn = "https://github.com/ericnost/gfw/tree/setup/shapefiles/Pacific", layer = "WCPFC") 
plot(wcpfc_map, border="wheat3", col="wheat2", add=T)

# SPC shapefile from: http://www.fao.org/geonetwork/srv/en/main.home?uuid=cc7dbf20-1b8b-11dd-8bbb-0017f293bd28
spc <- readOGR(dsn="https://raw.githubusercontent.com/ericnost/gfw/setup/data/spc.geojson")
plot(spc, border="wheat3", col="wheat3", add=T)

ERROR: Error in read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/countries.geojson"): could not find function "read_sf"


## Load in the attribute data on vessels from GFW and WCPFC

In [11]:
# Load GFW's main vessel database described here: https://globalfishingwatch.org/datasets-and-code/vessel-identity/
vessels_full <- read_csv("https://raw.githubusercontent.com/ericnost/gfw/setup/data/vessels.csv")

# Load the WCPFC database of Registered Fishing Vessels from: https://www.wcpfc.int/record-fishing-vessel-database
wcpfc <- read_csv("https://raw.githubusercontent.com/ericnost/gfw/setup/data/RFV_database_standardized.csv")

Parsed with column specification:
cols(
  mmsi = col_integer(),
  shipname = col_character(),
  callsign = col_character(),
  flag = col_character(),
  imo = col_character(),
  registry_geartype = col_character(),
  inferred_geartype = col_character(),
  inferred_geartype_score = col_double(),
  inferred_subgeartype = col_character(),
  inferred_subgeartype_score = col_double(),
  registry_length = col_double(),
  inferred_length = col_double(),
  registry_tonnage = col_double(),
  inferred_tonnage = col_double(),
  registry_engine_power = col_double(),
  inferred_engine_power = col_double(),
  source = col_character()
)
Parsed with column specification:
cols(
  .default = col_character(),
  VID = col_integer(),
  `Built in Year` = col_integer(),
  Crew = col_integer(),
  Length = col_double(),
  `Moulded Depth` = col_double(),
  Beam = col_double(),
  Tonnage = col_number(),
  `Engine Power` = col_integer(),
  `FishHold Capacity` = col_integer(),
  `IMO-LR` = col_integer()
)
See spec(

## Show what the WCPFC's Registry of Fishing Vessels looks like
It's a big database, so we'll just focus on the first 10 or so entries (vessels)

In [12]:
head(wcpfc)

Submitted by CCM,VID,Vessel Name,Flag,Registration Number,WIN,Previous Names,Reg Port,Owner Name,Owner Address,⋯,Auth Period To,Purse seine vessel authorised to tranship at sea,IMO-LR,Authorisation to tranship on the high seas,Under Charter,Charterer Name,Chartering CCM,Address of Charter,Charter Start Date,Charter End Date
Panama,11779,ANGEL NO. 2,Panama,50267-19,HPTG,JUPITER,Panama,ALL PERFECTION LIMITED,"VISTRA CORPORATE SERVICES CENTRE, WICKHAMS CAY II, ROAD TOWN, TORTOLA VG1110, BRITISH VIRGIN ISLANDS",⋯,31-Jan-21,Not specified,9146027,YES,NO,NA,NA,NA,NA,NA
Cook Islands,8723,LADY ANN II,Cook Islands,2118,E5U3018,KONA WIND,Avatiu,Ann Fishing Company Limited,"C/o- Cook Islands Trust and Banking Corporation Limited, Avarua, Rarotonga, Cook Islands.",⋯,31-Dec-20,NO,8854342,NO,NO,NA,NA,NA,NA,NA
China,11120,FU YUAN YU 7082,China,(MIN)CHUANDENG(JI)(2017)FT-100033,BZ2UT,FU YUAN YU 864,Fuzhou,"Fujian Zhongji Farsea Fishery Co., Ltd.","Jinyu Industrial District, Shenhu Town, Quanzhou Jinjiang City, Fujian Province",⋯,31-Mar-19,Not specified,8685583,YES,NO,NA,NA,NA,NA,NA
China,11122,FU YUAN YU 7085,China,(MIN)CHUANDENG(JI)(2017)FT-100035,BZ4UT,FU YUAN YU 866,Fuzhou,"Fujian Zhongji Farsea Fishery Co., Ltd.","Jinyu Industrial District, Shenhu Town, Quanzhou Jinjiang City, Fujian Province",⋯,31-Mar-19,Not specified,8685600,YES,NO,NA,NA,NA,NA,NA
China,11123,FU YUAN YU 7086,China,(MIN)CHUANDENG(JI)(2017)FT-100036,BZ5UT,FU YUAN YU 867,Fuzhou,"Fujian Zhongji Farsea Fishery Co., Ltd.","Jinyu Industrial District, Shenhu Town, Quanzhou Jinjiang City, Fujian Province",⋯,31-Mar-19,Not specified,8685612,YES,NO,NA,NA,NA,NA,NA
China,11124,FU YUAN YU 7087,China,(MIN)CHUANDENG(JI)(2017)FT-100037,BZ6UT,FU YUAN YU 868,Fuzhou,"Fujian Zhongji Farsea Fishery Co., Ltd.","Jinyu Industrial District, Shenhu Town, Quanzhou Jinjiang City, Fujian Province",⋯,31-Mar-19,Not specified,8685624,YES,NO,NA,NA,NA,NA,NA


## Zoom in to a selected company 

In [13]:
# For now, you have to type out the name of the company you want to focus on...exactly as it is spelled in the database.
# The default option is: Dongwon.

#ky<-wcpfc[grep("Kyokuyo", wcpfc$`Owner Name`),]
dongwon<-wcpfc[grep("Dongwon", wcpfc$`Owner Name`),] # Instead of "Dongwon" you would write some other company that appears in the Owner Name column.
dongwon

# Optional: Merge dongwon and ky to look at both of these 'keystone actors' simultaneously
#dongwon<-rbind(dongwon,ky)

Submitted by CCM,VID,Vessel Name,Flag,Registration Number,WIN,Previous Names,Reg Port,Owner Name,Owner Address,⋯,Auth Period To,Purse seine vessel authorised to tranship at sea,IMO-LR,Authorisation to tranship on the high seas,Under Charter,Charterer Name,Chartering CCM,Address of Charter,Charter Start Date,Charter End Date
Korea (Republic of),2129,ATUN TRES,Korea (Republic of),9512030-6260004,DTAC5,NONE,Busan,Dongwon,"DONGWON INDUSTRIES CO., LTD. /68, Mabang-ro, Seocho-gu, Seoul, Korea",⋯,18-Dec-20,Not applicable,8821553,YES,NO,NA,NA,NA,NA,NA
Korea (Republic of),10364,BADARO,Korea (Republic of),JJR-141009,D7MZ,BADARO,Jeju,Dongwon,"LTD/NO.275, YANGJAE-DONG, SEOCHO-GU, SEOUL, KOREA",⋯,24-Mar-20,No,9163439,YES,NO,NA,NA,NA,NA,NA
Korea (Republic of),1667,BLUE OCEAN,Korea (Republic of),0805002-6261103,DTBU5,NONE,Busan,Dongwon,"68, MABANG-RO, SEOCHO-GU, SEOUL, KOREA",⋯,24-Jun-18,No,9509396,No,NO,NA,NA,NA,NA,NA
Korea (Republic of),11854,BONAMI,Korea (Republic of),1909001-6261103,6KSG,NONE,Busan,Dongwon,"68 MABANG-RO, SEOCHO-GU, SEOUL, KOREA",⋯,15-Sep-24,NO,9848869,NO,NO,NA,NA,NA,NA,NA
Korea (Republic of),2241,CARIBE,Korea (Republic of),0407001-6261108,DTBQ7,NONE,Busan,Dongwon,"68, MABANG-RO, SEOCHO-GU, SEOUL, KOREA",⋯,20-Jul-24,No,8102854,No,NO,NA,NA,NA,NA,NA
Korea (Republic of),1663,COSMOS KIM,Korea (Republic of),9510055-6260009,HLLF,NONE,Busan,Dongwon,"68, MABANG-RO, SEOCHO-GU, SEOUL, KOREA",⋯,08-May-21,No,8103092,No,NO,NA,NA,NA,NA,NA
Korea (Republic of),1671,DEOLINDA,Korea (Republic of),9510072-6260008,6NHD,NONE,Busan,Dongwon,"68, MABANG-RO, SEOCHO-GU, SEOUL, KOREA",⋯,07-Nov-20,No,7827469,No,NO,NA,NA,NA,NA,NA
Korea (Republic of),1670,ELSPETH,Korea (Republic of),9510073-6260007,6NHK,NONE,Busan,Dongwon,"68, MABANG-RO, SEOCHO-GU, SEOUL, KOREA",⋯,19-Jul-20,No,7827483,No,NO,NA,NA,NA,NA,NA
Korea (Republic of),11383,HANARA,Korea (Republic of),1602001-6261103,6KAP,NONE,Busan,Dongwon,"68, MABANG-RO, SEOCHO-GU, SEOUL, KOREA",⋯,10-Feb-21,NO,9743435,NO,NO,NA,NA,NA,NA,NA
Korea (Republic of),4593,JANG BO GO,Korea (Republic of),0810001-6261103,DTBU8,NONE,Busan,Dongwon,"68, MABANG-RO, SEOCHO-GU, SEOUL, KOREA",⋯,02-Nov-23,No,9511301,No,NO,NA,NA,NA,NA,NA


## Get the identifying info for this company's vessels, as described in the WCPFC RFV.


In [14]:
dongwon_imos<-dongwon$`IMO-LR`
dongwon_ircs<-dongwon$IRCS

filtered<-vessels_full[which(vessels_full$imo %in% dongwon_imos | vessels_full$callsign %in% dongwon_ircs ),]

dailies_filtered<-daily[which(daily$mmsi %in% filtered$mmsi),] # This company's vessels
dailies_extracted<-daily[-which(daily$mmsi %in% filtered$mmsi),] # All other vessels

## Join vessel ID information and fishing effort data


In [16]:
dongwon_fishing<-merge(dailies_filtered, filtered, by = "mmsi") # This company's vessels
allother_fishing<-merge(dailies_extracted, vessels_full, by ="mmsi") # All other vessels
dongwon_fishing

mmsi,date,lat_bin,lon_bin,fishing_hours,lat,long,latlong,shipname,callsign,⋯,inferred_geartype_score,inferred_subgeartype,inferred_subgeartype_score,registry_length,inferred_length,registry_tonnage,inferred_tonnage,registry_engine_power,inferred_engine_power,source
440195000,2016-12-22,-20,1741,0.4751389,-1.95,174.15,"-1.95, 174.15",SEGYERO,6KCD2,⋯,0.8005054,purse_seines,0.8005054,79.6000,76.38455,NA,1571.5611,3667.991,4028.1910,"AIS,CLAV,FFA,ISSF,ITU,KOR,WCPFC"
440233000,2016-12-22,-68,-1755,0.6090278,-6.75,-175.45,"-6.75, -175.45",CARIBE,DTBQ7,⋯,0.9984821,purse_seines,0.9984821,74.6630,67.21381,NA,1273.2418,2672.400,3079.8755,"AIS,CLAV,FFA,ISSF,ITU,KOR,WCPFC"
440233000,2016-12-22,-68,-1756,2.5755556,-6.75,-175.55,"-6.75, -175.55",CARIBE,DTBQ7,⋯,0.9984821,purse_seines,0.9984821,74.6630,67.21381,NA,1273.2418,2672.400,3079.8755,"AIS,CLAV,FFA,ISSF,ITU,KOR,WCPFC"
440233000,2016-12-22,-67,-1756,0.3052778,-6.65,-175.55,"-6.65, -175.55",CARIBE,DTBQ7,⋯,0.9984821,purse_seines,0.9984821,74.6630,67.21381,NA,1273.2418,2672.400,3079.8755,"AIS,CLAV,FFA,ISSF,ITU,KOR,WCPFC"
440233000,2016-12-22,-64,-1759,0.3650000,-6.35,-175.85,"-6.35, -175.85",CARIBE,DTBQ7,⋯,0.9984821,purse_seines,0.9984821,74.6630,67.21381,NA,1273.2418,2672.400,3079.8755,"AIS,CLAV,FFA,ISSF,ITU,KOR,WCPFC"
440236000,2016-12-22,81,-183,6.8979167,8.15,-18.25,"8.15, -18.25",DONGWON216,6NMK,⋯,0.9999995,drifting_longlines,0.9999995,52.6204,47.79229,408.2,449.4847,890.800,873.2205,"AIS,CLAV,FFA,IATTC,ITU,KOR,WCPFC"
440479000,2016-12-22,-20,1740,0.8443056,-1.95,174.05,"-1.95, 174.05",ELSPETH,6NHK,⋯,0.9989618,purse_seines,0.9989618,73.0490,70.49200,NA,1192.8457,2672.733,2947.6311,"AIS,CLAV,FFA,ISSF,ITU,KOR,WCPFC"
440479000,2016-12-22,-20,1741,0.3725000,-1.95,174.15,"-1.95, 174.15",ELSPETH,6NHK,⋯,0.9989618,purse_seines,0.9989618,73.0490,70.49200,NA,1192.8457,2672.733,2947.6311,"AIS,CLAV,FFA,ISSF,ITU,KOR,WCPFC"
440479000,2016-12-22,-19,1741,0.7208333,-1.85,174.15,"-1.85, 174.15",ELSPETH,6NHK,⋯,0.9989618,purse_seines,0.9989618,73.0490,70.49200,NA,1192.8457,2672.733,2947.6311,"AIS,CLAV,FFA,ISSF,ITU,KOR,WCPFC"
440574000,2016-12-22,-27,-1574,0.4111111,-2.65,-157.35,"-2.65, -157.35",DONGWON208,6NIM,⋯,1.0000000,drifting_longlines,1.0000000,51.1420,49.84720,408.2,485.2673,891.900,1016.2043,"AIS,CLAV,FFA,IATTC,ITU,KOR,WCPFC,REVIEW"


## Aggregate fishing effort information for all other vessels, by lat/lon
What we are trying to do is show how much fishing occurred in each location by each country (flag), apart from the vesssels owned by this company.

In the first set of outputs, you will see for each location ("lat/lon bin") the total number of fishing hours recorded, the number of different flags found, and the number of different vessels counted ("count")

In the second set, you will see for each location, the flag that fished the most there, and the total number of hours fished.

In [17]:
aggregated<-allother_fishing %>%
  group_by(lat_bin, lon_bin) %>% #, flag # Aggregated at specific lat lon bins
  summarise(totalFishingHours=sum(fishing_hours), flags=n_distinct(flag), count=n_distinct(mmsi)) 
# aggregated = total fishing effort at this lat lon, number of unique flags and mmsis at this lat lon

aggregated$lat_bin<-(aggregated$lat_bin/10)+.05
aggregated$lon_bin<-(aggregated$lon_bin/10)+.05

# Produce a set of points with the most productive (in terms of hours of fishing effort) vessel
# for each lat/lon bin and report its flag
aggregated_max<-allother_fishing %>%
  group_by(lat_bin, lon_bin, flag) %>% # Aggregated at specific lat long bins and flags
  summarise(totalFishingHours=sum(fishing_hours)) 
aggregated_max<-aggregated_max %>% 
  group_by(lat_bin, lon_bin) %>%
  filter(totalFishingHours == max(totalFishingHours))  #max here 553 -365
# aggregated_max = total fishing effort at this lat long and the country that did it

aggregated_max$lat_bin<-(aggregated_max$lat_bin/10)+.05
aggregated_max$lon_bin<-(aggregated_max$lon_bin/10)+.05

aggregated
aggregated_max

lat_bin,lon_bin,totalFishingHours,flags,count
-77.75,178.45,1.5980556,1,1
-77.65,178.45,4.1262500,1,1
-77.55,178.45,2.4544444,1,1
-77.45,178.45,4.1748611,1,1
-77.35,178.45,0.7490278,1,1
-77.25,179.25,0.6945833,1,1
-77.25,179.35,1.6205556,1,1
-77.15,179.35,3.2358333,1,1
-75.95,-171.85,1.5927778,1,1
-75.95,-171.65,1.6194444,1,1


lat_bin,lon_bin,flag,totalFishingHours
-77.75,178.45,UKR,1.5980556
-77.65,178.45,UKR,4.1262500
-77.55,178.45,UKR,2.4544444
-77.45,178.45,UKR,4.1748611
-77.35,178.45,UKR,0.7490278
-77.25,179.25,UKR,0.6945833
-77.25,179.35,UKR,1.6205556
-77.15,179.35,UKR,3.2358333
-75.95,-171.85,RUS,1.5927778
-75.95,-171.65,RUS,1.6194444


## Do the same thing, but for the selected company (Dongwon)

In [21]:
# Aggregate this company's vessels by lat/lon

dongwon_aggregated<-dongwon_fishing %>%
  group_by(lat_bin, lon_bin) %>% #aggregated at specific lat long bins
  summarise(totalFishingHours=sum(fishing_hours), flag="COMP", count=n_distinct(mmsi)) 
# Show total fishing effort at this lat long, number of unique Dongwon mmsis at this lat long
dongwon_aggregated$lat_bin<-(dongwon_aggregated$lat_bin/10)+.05
dongwon_aggregated$lon_bin<-(dongwon_aggregated$lon_bin/10)+.05

dongwon_aggregated

lat_bin,lon_bin,totalFishingHours,flag,count
-14.85,45.95,1.6547222,COMP,1
-14.75,45.85,1.8319444,COMP,1
-14.75,46.15,1.1752778,COMP,1
-14.65,45.75,2.9165278,COMP,1
-14.55,45.75,0.8665278,COMP,1
-14.55,45.95,0.9012500,COMP,1
-14.55,46.15,0.7481944,COMP,1
-14.45,45.95,2.0776389,COMP,1
-14.45,46.05,0.6525000,COMP,1
-14.45,46.15,0.7959722,COMP,1


## Bringing it all together - for each location, was this company the biggest fisher or was it a specific flag/fleet?

Again, you will see for each location, the flag that fished the most there, and the total number of hours fished. If the flag is "COMP", that means the selected company did the most fishing in that location.

In [22]:
# Now, for each lat/lon, we find the biggest fisher - either this company or other vessels
# We will show the biggest flag or company (Dongwon) for each lat/lon.

dongwon_aggregated<-dongwon_aggregated[,-c(5)]
colnames(dongwon_aggregated)<-c("lat_bin", "lon_bin", "totalFishingHours", "flag")
total<-rbind(aggregated_max, dongwon_aggregated)
total<-total %>% 
  group_by(lat_bin, lon_bin) %>%
  filter(totalFishingHours == max(totalFishingHours)) 

total

lat_bin,lon_bin,flag,totalFishingHours
-77.75,178.45,UKR,1.5980556
-77.65,178.45,UKR,4.1262500
-77.55,178.45,UKR,2.4544444
-77.45,178.45,UKR,4.1748611
-77.35,178.45,UKR,0.7490278
-77.25,179.25,UKR,0.6945833
-77.25,179.35,UKR,1.6205556
-77.15,179.35,UKR,3.2358333
-75.95,-171.85,RUS,1.5927778
-75.95,-171.65,RUS,1.6194444


## Export this analysis to spreadsheets

In [ ]:
# Running this cell will save the data we have processed and analyzed so far. You may have to click on the folder
# icon and hit refresh to see it. You can then click on the inverted ... next to each file and download them.

write_csv(aggregated, "all_other_fishing_total.csv")
write_csv(aggregated_max, "all_other_fishing_max.csv")
write_csv(dongwon_aggregated, "dongwon.csv")
write_csv(total, "all_fishing_max.csv")

## Simplify the data, for the purpose of mapping it

In [23]:
# Since there are many flags that fish in this part of the world, and because of the limitations of this platform when
# it comes to data visualization and mapping, we need to aggregate our data a bit more. This will cause our analysis to 
# lose some of its power, but for the full paper, rather than this demo, we probably do NOT need to do this.

total <- mutate(total, flag=recode(flag, 
                         "COMP"="COMPANY", # We'll group all of this company's vessels
                         "CHN"="CHINA", # We'll compare to China-flagged vessels
                         .default="OTHR")) # All otherly-flagged vessels will be lumped together

total

lat_bin,lon_bin,flag,totalFishingHours
-77.75,178.45,OTHR,1.5980556
-77.65,178.45,OTHR,4.1262500
-77.55,178.45,OTHR,2.4544444
-77.45,178.45,OTHR,4.1748611
-77.35,178.45,OTHR,0.7490278
-77.25,179.25,OTHR,0.6945833
-77.25,179.35,OTHR,1.6205556
-77.15,179.35,OTHR,3.2358333
-75.95,-171.85,OTHR,1.5927778
-75.95,-171.65,OTHR,1.6194444


## Create spatial point data out of the information we've been working with above.

In [ ]:
total_pts = st_as_sf(total, coords = c("lon_bin", "lat_bin"), crs = 4326)

## Standardize the Coordinate Reference System across our shapefiles
We just need to make sure that each of our pieces of geographic data are in the same format.

In [ ]:
#st_transform(total, crs = 4326)
wcpfc_map<-st_transform(wcpfc_map, crs = 4326)
spc<-st_transform(spc, crs = 4326)
countries<-st_transform(countries, crs = 4326)

## "Clip" the fishing data we processed above to focus only on The Pacific Community (SPC)

In [ ]:
clipped_total<-st_intersection(total_pts, st_buffer(spc, 0))
clipped_total

## Project the spatial data
We have to project the data to minimize distortions in taking 3D real-world locations and visualizing them on our 2D flat screens

In [ ]:
p = CRS("+proj=aea +lat_0=-30 +lat_1=30 +lon_0=-180")
wcpfc_map<-st_transform(wcpfc_map, p)
spc<-st_transform(spc, p)
countries<-st_transform(countries, p)
clipped_total<-st_transform(clipped_total, p)

In [ ]:
#DEVELOP COLOR SCALE, CHANGE FROM + to POINTS
#library(RColorBrewer) # creates nice color schemes
#library(classInt) 
#nclr <- 4
#plotclr <- brewer.pal(nclr,"BuPu")
#class <- classIntervals(clipped_total$totalFishingHours, nclr, style="quantile") #clipped #dongwon_clipped
#colcode <- findColours(class, plotclr)
#Create a custom color scale
library(RColorBrewer)
myColors <- brewer.pal(3,"Set1")
names(myColors) <- levels(clipped_total$flag)
colScale <- scale_colour_manual(name = "grp",values = myColors)



#plot(countries, border="wheat3", col="wheat1")
#plot(wcpfc_map, border="wheat3", col="lightblue")
#plot(spc, border="wheat3", col="red", add=T)
#plot(clipped_total["flag"], pal = sf.colors(3, categorical = TRUE), 
#     pch=16, key.pos = 4, key.width = lcm(4.5), axes = TRUE, graticule = TRUE)
#plot
map = ggplot() + 
geom_sf(data = wcpfc_map, fill = "blue", alpha=.1) +
geom_sf(data = spc, fill = "green", alpha=.1) +
#geom_sf(data = countries, aes(fill = NAME)) +
geom_point(
    data=clipped_total,
    aes(color = flag, geometry = geometry),
    stat = "sf_coordinates"
  ) +
#geom_sf(data = clipped_total, aes(fill=clipped_total$flag)) +
colScale
#scale_size_continuous(range=c(1,12))
#theme(legend.position = "none") 
map

In [ ]:
# Finally, we can look at t
clipped_total_flag <- clipped_total%>%
  group_by(flag) %>%
  summarise(totalFishingHours=sum(totalFishingHours))
clipped_total_flag